In [1]:
#!pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 13.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 15.7 MB 67.1 MB/s 
     |████████████████████████████████| 11.3 MB 61.6 MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=52883b5e0b729661f7ecd20061d3f92c3811fa18f50c7af1dc3d4e8e41067d7f
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take i

In [1]:
import os
import pandas as pd
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import plotly.graph_objs as go
import gensim
from gensim import corpora, models, similarities
import logging
import tempfile
from nltk.corpus import stopwords
from string import punctuation
from collections import OrderedDict
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

init_notebook_mode(connected=True) #do not miss this line

import warnings
warnings.filterwarnings("ignore")

In [2]:
datafile = 'data_elonmusk.csv'

In [3]:
import pandas as pd
tweets = pd.read_csv(datafile, encoding='latin1')
tweets = tweets.assign(Time=pd.to_datetime(tweets.Time)).drop('row ID', axis='columns')

print("Number of tweets: ",len(tweets['Tweet']))
tweets.head(5)

Number of tweets:  3218


,Tweet,Time,Retweet from,User
0,@MeltingIce Assuming max acceleration of 2 to ...,2017-09-29 17:39:19,NaN,elonmusk
1,RT @SpaceX: BFR is capable of transporting sat...,2017-09-29 10:44:54,SpaceX,elonmusk
2,@bigajm Yup :),2017-09-29 10:39:57,NaN,elonmusk
3,Part 2 https://t.co/8Fvu57muhM,2017-09-29 09:56:12,NaN,elonmusk
4,Fly to most places on Earth in under 30 mins a...,2017-09-29 09:19:21,NaN,elonmusk


***tweets***

In [4]:
# Preparing a corpus for analysis and checking first 10 entries

corpus=[]
a=[]
for i in range(len(tweets['Tweet'])):
        a=tweets['Tweet'][i]
        corpus.append(a)
        
corpus[0:10]

["@MeltingIce Assuming max acceleration of 2 to 3 g's, but in a comfortable direction. Will feel like a mild to moder? https://t.co/fpjmEgrHfC",
 'RT @SpaceX: BFR is capable of transporting satellites to orbit, crew and cargo to the @Space_Station and completing missions to the Moon an?',
 '@bigajm Yup :)',
 'Part 2 https://t.co/8Fvu57muhM',
 'Fly to most places on Earth in under 30 mins and anywhere in under 60. Cost per seat should be? https://t.co/dGYDdGttYd',
 'RT @SpaceX: Supporting the creation of a permanent, self-sustaining human presence on Mars. https://t.co/kCtBLPbSg8 https://t.co/ra6hKsrOcG',
 'BFR will take you anywhere on Earth in less than 60 mins https://t.co/HWt9BZ1FI9',
 'Mars City\nOpposite of Earth. Dawn and dusk sky are blue on Mars and day sky is red. https://t.co/XHcZIdgqnb',
 'Moon Base Alpha https://t.co/voY8qEW9kl',
 "Will be announcing something really special at today's talk https://t.co/plXTBJY6ia"]

In [5]:
TEMP_FOLDER = tempfile.gettempdir()
print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Folder "/var/folders/bd/8sj691g54txb24r7swvnh99h0000gn/T" will be used to save temporary dictionary and corpus.


In [6]:
import nltk
nltk.download('stopwords')
  

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/LeonardoLins/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
# removing common words and tokenizing
list1 = ['RT','rt']
stoplist = stopwords.words('english') + list(punctuation) + list1

texts = [[word for word in str(document).lower().split() if word not in stoplist] for document in corpus]

dictionary = corpora.Dictionary(texts)
dictionary.save(os.path.join(TEMP_FOLDER, 'elon.dict'))  # store the dictionary, for future reference

#print(dictionary)
#print(dictionary.token2id)

2022-02-23 18:28:46,965 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-02-23 18:28:47,064 : INFO : built Dictionary(11842 unique tokens: ['2', '3', '@meltingice', 'acceleration', 'assuming']...) from 3218 documents (total 30256 corpus positions)
2022-02-23 18:28:47,065 : INFO : saving Dictionary object under /var/folders/bd/8sj691g54txb24r7swvnh99h0000gn/T/elon.dict, separately None
2022-02-23 18:28:47,072 : INFO : saved /var/folders/bd/8sj691g54txb24r7swvnh99h0000gn/T/elon.dict


In [8]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'elon.mm'), corpus)  # store to disk, for later use

2022-02-23 18:28:50,084 : INFO : storing corpus in Matrix Market format to /var/folders/bd/8sj691g54txb24r7swvnh99h0000gn/T/elon.mm
2022-02-23 18:28:50,086 : INFO : saving sparse matrix to /var/folders/bd/8sj691g54txb24r7swvnh99h0000gn/T/elon.mm
2022-02-23 18:28:50,086 : INFO : PROGRESS: saving document #0
2022-02-23 18:28:50,121 : INFO : PROGRESS: saving document #1000
2022-02-23 18:28:50,148 : INFO : PROGRESS: saving document #2000
2022-02-23 18:28:50,180 : INFO : PROGRESS: saving document #3000
2022-02-23 18:28:50,189 : INFO : saved 3218x11842 matrix, density=0.079% (29959/38107556)
2022-02-23 18:28:50,190 : INFO : saving MmCorpus index to /var/folders/bd/8sj691g54txb24r7swvnh99h0000gn/T/elon.mm.index


Nas células anteriores, criei um corpus de documentos representados como um fluxo de vetores. Para continuar, vamos usar esse corpus, com a ajuda do Gensim.

###Criação de uma transformação

As transformações são objetos Python padrão, normalmente inicializados por meio de um corpus de treinamento:

Diferentes transformações podem exigir diferentes parâmetros de inicialização; no caso do TfIdf, o "treinamento" consiste simplesmente em
percorrer o corpus fornecido uma vez e computar as frequências de documentos de todos os seus recursos.
Treinar outros modelos, como Análise Semântica Latente ou Alocação de Dirichlet Latente, é muito mais envolvente e,
conseqüentemente, leva muito mais tempo.

In [9]:
tfidf = models.TfidfModel(corpus) # step 1 -- initialize a model

2022-02-23 18:28:54,424 : INFO : collecting document frequencies
2022-02-23 18:28:54,426 : INFO : PROGRESS: processing document #0
2022-02-23 18:28:54,440 : INFO : calculating IDF weights for 3218 documents and 11841 features (29959 matrix non-zeros)


### Nota
As transformações sempre são convertidas entre dois espaços vetoriais específicos. O mesmo espaço vetorial (= o mesmo conjunto de ids de recursos) deve ser usado para treinamento, bem como para transformações vetoriais subsequentes. Deixar de usar o mesmo espaço de recurso de entrada, como aplicar um pré-processamento de string diferente, usar IDs de recurso diferentes ou usar vetores de entrada de pacote de palavras onde vetores TfIdf são esperados, resultará em incompatibilidade de recurso durante chamadas de transformação e, consequentemente, em qualquer lixo saída e / ou exceções de tempo de execução.

A partir de agora, tfidf é tratado como um objeto somente leitura que pode ser usado para aplicar uma transformação a um corpus inteiro:

In [10]:
corpus_tfidf = tfidf[corpus]  # step 2 -- use the model to transform vectors

### LDA:


LDA somos experts

In [11]:
total_topics = 5

In [12]:
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=total_topics)
corpus_lda = lda[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

2022-02-23 18:29:00,040 : INFO : using symmetric alpha at 0.2
2022-02-23 18:29:00,042 : INFO : using symmetric eta at 0.2
2022-02-23 18:29:00,048 : INFO : using serial LDA version on this node
2022-02-23 18:29:00,062 : INFO : running online (single-pass) LDA training, 5 topics, 1 passes over the supplied corpus of 3218 documents, updating model once every 2000 documents, evaluating perplexity every 3218 documents, iterating 50x with a convergence threshold of 0.001000
2022-02-23 18:29:00,063 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2022-02-23 18:29:00,064 : INFO : PROGRESS: pass 0, at document #2000/3218
2022-02-23 18:29:02,464 : INFO : merging changes from 2000 documents into a model of 3218 documents
2022-02-23 18:29:02,482 : INFO : topic #0 (0.200): 0.011*"model" + 0.007*"3" + 0.005*"tesla" + 0.004*"like" + 0.003*"launch" + 0.003*"yes" + 0.003*"@teslamotors" + 0.003*"want" + 0.003*"good" + 0.

In [13]:
#Show first n important word in the topics:
lda.show_topics(total_topics,5)

[(0,
  '0.011*"model" + 0.007*"tesla" + 0.006*"w" + 0.005*"rocket" + 0.005*"@elonmusk"'),
 (1,
  '0.010*"tesla" + 0.010*"model" + 0.006*"like" + 0.005*"..." + 0.004*"good"'),
 (2,
  '0.012*"tesla" + 0.005*"model" + 0.004*"test" + 0.004*"car" + 0.003*"would"'),
 (3,
  '0.004*"@spacex" + 0.004*"tesla" + 0.004*"thanks" + 0.004*"rocket" + 0.003*"next"'),
 (4,
  '0.011*"launch" + 0.011*"falcon" + 0.009*"@spacex:" + 0.008*"9" + 0.007*"tesla"')]

In [14]:
data_lda = {i: OrderedDict(lda.show_topic(i,25)) for i in range(total_topics)}
#data_lda

In [15]:
df_lda = pd.DataFrame(data_lda)
df_lda = df_lda.fillna(0).T
print(df_lda.shape)

(5, 85)


In [16]:
df_lda

,model,tesla,w,rocket,@elonmusk,like,new,launch,@teslamotors,flight,...,:),attempt,still,video,orbit,go,last,hyperloop,droneship,space
0,0.010885,0.007058,0.005742,0.005298,0.004834,0.004738,0.004433,0.004170,0.003474,0.003218,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.010020,0.010173,0.000000,0.000000,0.000000,0.006326,0.000000,0.000000,0.002876,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.004832,0.012110,0.000000,0.002254,0.000000,0.000000,0.002923,0.000000,0.001877,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.002191,0.003850,0.000000,0.003732,0.002837,0.002300,0.000000,0.002516,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.006768,0.000000,0.006063,0.000000,0.000000,0.000000,0.010615,0.000000,0.000000,...,0.003137,0.002605,0.002428,0.002369,0.002366,0.002048,0.002006,0.001942,0.001884,0.001874


In [17]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
pyLDAvis.__version__

'3.3.1'

In [18]:

panel =gensimvis.prepare(lda, corpus_lda, dictionary, mds='tsne')
panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
4      150.222153  -72.269554       1        1  41.278685
0       51.163364 -159.968018       2        1  20.559767
1      -73.144554   21.523815       3        1  17.693264
3      -66.410133 -110.606438       4        1  10.246448
2       54.417324   12.443444       5        1  10.221836, topic_info=              Term       Freq      Total Category  logprob  loglift
580          model  16.000000  16.000000  Default  30.0000  30.0000
231          tesla  24.000000  24.000000  Default  29.0000  29.0000
256              9  11.000000  11.000000  Default  28.0000  28.0000
407              w   4.000000   4.000000  Default  27.0000  27.0000
10            like   8.000000   8.000000  Default  26.0000  26.0000
...            ...        ...        ...      ...      ...      ...
90          rocket   0.741573  14.031013   Topic5  -6.0949  -0.6596
1423          cape   0.562591   2.362744   Topic5  -6.3711   0.8456
342         coming   0.578740   3.223474   Topic5  -6.3428   0.5633
1347  @teslamotors   0.617345   6.650134   Topic5  -6.2782  -0.0963
328        landing   0.623593   9.202336   Topic5  -6.2681  -0.4111

[368 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1013      1  0.655448  #dragon
1013      4  0.655448  #dragon
2277      2  0.620273       --
272       1  0.319114      ...
272       2  0.159557      ...
...     ...       ...      ...
884       3  0.428042    yeah,
144       1  0.287404     yes,
144       2  0.287404     yes,
144       3  0.287404     yes,
2695      3  1.604613     yet.

[322 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 2, 4, 3])